# Evaluating ORA (gpt-4o) and base gpt-4o with MATH

### Configuration

In [2]:
def print_answer(response):
    print("Thoughts:")
    print("\n".join(response["thoughts"]))
    print("Answer:")
    print(response["answer"])

In [3]:
from openai import OpenAI
import os

In [4]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
def call_strong(messages: list) -> str:
    completion = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    return completion.choices[0].message.content

def call_mini(messages: list) -> str:
    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [6]:
from reasonable import DefaultReasoningAgent

In [7]:
agent = DefaultReasoningAgent(
    main_function=call_strong,
    thoughts_function=call_strong,
    max_steps=10,
    verbose=True
)

ReasoningAgent initialized. Verbose mode is on.


In [8]:
import re

def evaluate_llm_response(question, llm_answer_func, evaluator_func):
    question_text = question["question"]
    correct_solution = question["solution"]
    correct_answer = question["answer"]

    llm_answer = llm_answer_func([{"role": "user", "content": question_text}])

    evaluation_prompt = (
        f"Question: {question_text}\n"
        f"Correct solution: {correct_solution}\n"
        f"Correct answer: {correct_answer}\n"
        f"Answer by student: {llm_answer}\n"
        "Please evaluate the correctness of the student's answer above."
        "Evaluate the student's answer in <evaluation> tag and provide a verdict ('correct' or 'incorrect') in <verdict> tag and score (from 0 to 10) in <score> tag."
        "Respond in this format:"
        "<evaluation>"
        "..."
        "</evaluation>"
        "<verdict>"
        "..."
        "</verdict>"
        "<score>"
        "..."
        "</score>"
    )

    evaluation_result = evaluator_func([{"role": "user", "content": evaluation_prompt}])

    # extract score
    pattern = r'<score>(.*?)</score>'
    score = re.search(pattern, evaluation_result, re.DOTALL)
    if score:
        score = int(score.group(1).strip())
    else:
        score = 0

    # extract verdict
    pattern = r'<verdict>(.*?)</verdict>'
    verdict = re.search(pattern, evaluation_result, re.DOTALL)
    if verdict:
        verdict = verdict.group(1).strip()
        if "incorrect" in verdict:
            verdict = 0
        else:
            verdict = 1
    else:
        verdict = 0

    return {"question": question_text, "correct_answer": correct_answer, "llm_answer": llm_answer, "score": score, "verdict": verdict}

### Loading dataset

In [25]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/MATH-500")

Let's randomly choose 50 questions from dataset

In [9]:
import random

In [10]:
MATH50_ds = []
for example in ds["test"]:
    MATH50_ds.append({
        "question": example["problem"],
        "solution": example["solution"],
        "answer": example["answer"]
    })

In [11]:
random.shuffle(MATH50_ds)

In [12]:
MATH50_ds = MATH50_ds[:50]

In [13]:
MATH50_ds

[{'question': 'Roslyn has ten boxes. Five of the boxes contain pencils, four of the boxes contain pens, and two of the boxes contain both pens and pencils. How many boxes contain neither pens nor pencils?',
  'solution': 'Of the 5 boxes with pencils, 2 have pens also, so $5-2=3$ have pencils only.  Similarly, $4-2 =2$ of the boxes have pens only:\n\n\n[asy]\nunitsize(0.05cm);\nlabel("Pencils", (2,74));\nlabel("Pens", (80,74));\ndraw(Circle((30,45), 22));\ndraw(Circle((58, 45), 22));\nlabel("$2$", (44, 45));\nlabel(scale(0.8)*"$3$",(28,58));\nlabel(scale(0.8)*"$2$",(63,58));\n[/asy]\n\nThat gives us $3+2+2=7$ boxes with pens, pencils, or both.  This leaves $10-7 = \\boxed{3}$ with neither.',
  'answer': '3'},
 {'question': 'What is the last nonzero digit to the right of the decimal point in the decimal expansion of $\\frac{137}{500}$?',
  'solution': 'We can rewrite $\\frac{137}{500}$ in the form $\\frac{274}{1000}$, so $\\frac{137}{500} = \\frac{274}{1000} = 0.274$ and the last nonzero

### Evaluating base model

In [14]:
from tqdm import tqdm

In [15]:
base_model_test_info = {
    "total": 0,
    "score": 0,
    "answers" : []
}

max_total = 50
max_score = 50 * 10

for question in tqdm(MATH50_ds):
    foo = evaluate_llm_response(question, call_strong, call_strong)
    base_model_test_info["total"] += foo["verdict"]
    base_model_test_info["score"] += foo["score"]
    base_model_test_info["answers"].append(foo)

    print(f"Current accuracy: {base_model_test_info['total']/len(base_model_test_info['answers'])}/1.0")
    print(f"Current score: {base_model_test_info['score']}/{10*len(base_model_test_info['answers'])}")

  2%|▏         | 1/50 [00:14<12:12, 14.94s/it]

Current accuracy: 1.0/1.0
Current score: 10/10


  4%|▍         | 2/50 [00:30<12:28, 15.59s/it]

Current accuracy: 1.0/1.0
Current score: 20/20


  6%|▌         | 3/50 [00:51<13:58, 17.85s/it]

...

Current accuracy: 0.75/1.0
Current score: 398/480


 98%|█████████▊| 49/50 [23:06<00:23, 23.82s/it]

Current accuracy: 0.7551020408163265/1.0
Current score: 408/490


100%|██████████| 50/50 [23:40<00:00, 28.42s/it]

Current accuracy: 0.74/1.0
Current score: 411/500


Saving results to json

In [17]:
import json

with open("base_model_test_info.json", 'w', encoding='utf-8') as json_file:
    json.dump(base_model_test_info, json_file, ensure_ascii=False, indent=4)

Saving questions to json

In [ ]:
with open("MATH50_ds.json", 'w', encoding='utf-8') as json_file:
    json.dump(MATH50_ds, json_file, ensure_ascii=False, indent=4)

Loading questions from json

In [10]:
import json

In [21]:
with open("base_model_test_info.json", 'r', encoding='utf-8') as json_file:
    base_model_test_info = json.loads(json_file.read())

In [ ]:
MATH50_ds = []
for example in base_model_test_info["answers"]:
    MATH50_ds.append(example)

In [35]:
new_MATH50_ds = []
for example in ds["test"]:
    if str(example["problem"]) in MATH50_ds:
        new_MATH50_ds.append({
            "question": example["problem"],
            "solution": example["solution"],
            "answer": example["answer"]
        })

In [37]:
len(new_MATH50_ds)

50